# Setting the fundarmental group of $\Sigma_{g,1}$ 

In [1]:
require './freegroup.rb'

true

## generators

In [2]:
genus = 3
alphabet = %w(a b c d e f)

Gens = [Letter.new] + alphabet.map{|x| Letter.new(x)}
Gens.freeze
p Gens[1..-1].map(&:show)

a = [Gens[0]] + Gens[1..-1].each_slice(2).map(&:first)
b = [Gens[0]] + Gens[1..-1].each_slice(2).map(&:last)
p a.map(&:show)
nil

["a", "b", "c", "d", "e", "f"]
["1", "a", "c", "e"]


In [3]:
comms = []
(Gens.size/2 + 1).times do |i|
  comms << Group.commutator(a[i], b[i])
end
p comms.map(&:show)

["1111", "abAB", "cdCD", "efEF"]


["1111", "abAB", "cdCD", "efEF"]

In [4]:
Rseq = []
(Gens.size/2).times do |i|
  Rseq += [a[i+1], b[i+1].inverse, a[i+1].inverse, b[i+1]]
end
Rseq.freeze
p Rseq.map(&:show) #.index('A')
nil

<main>:2: warning: already initialized constant Rseq
<main>:2: warning: already initialized constant Rseq
<main>:2: warning: previous definition of Rseq was here
<main>:2: warning: already initialized constant Rseq
<main>:2: warning: previous definition of Rseq was here


["a", "B", "A", "b", "c", "D", "C", "d", "e", "F", "E", "f"]


# The Algorithm

In [19]:
#myw = Word.new(a[1], b[2].inverse)
#myw = Word.new(a[1])^3
c = Word.new(a[1], b[1], a[1].inverse); myw = c
myw = comms[1]
#myw = Word.new(a[1])*(comms[2]^2)
#myw = Word.new(a[1])*comms[2]*c*comms[2]*(c.inverse)
p myw.show

"abAB"


"abAB"

In [20]:
box = {}
Rseq.each{|x| box.store(x.char,[])}

facs = myw.flatten.factors

facs.each_with_index do |c, i|
  k = i
  box[c.char].insert(-1, 2*k)
  box[c.inverse.char].insert(0,2*k+1)
end
p box, box.values.map(&:to_s).join
nil

{"a"=>[5, 0], "B"=>[3, 6], "A"=>[1, 4], "b"=>[7, 2], "c"=>[], "D"=>[], "C"=>[], "d"=>[], "e"=>[], "F"=>[], "E"=>[], "f"=>[]}
"[5, 0][3, 6][1, 4][7, 2][][][][][][][][]"


In [37]:
p myw.show
p box.values.map(&:to_s).join

bvf = box.values.flatten
size = bvf.size

odd_pairs=(1..size/2).to_a.map{|x| 2*x-1}.combination(2)
#odd_pairs = [[1,7],[3,5]]

coprods = odd_pairs.map do |pair|
  succs = [[pair[0], pair[0]+1], [pair[1], (pair[1]+1)%size]]
  order = bvf.select{|n| succs.flatten.include?(n)}
#  p "#{succs.to_s}, #{order.to_s}"

  # the parity of order
  parity = (order.index(pair[0]) - order.index(pair[0]+1) + 1).even? ? 'even' : 'odd'
  
  # the sign of order
  sign0 = (order.index(pair[0]) < order.index(pair[0]+1)) ? 1 : -1
#  sign1 = (idxs[1][0] > idxs[1][1]) ? 1 : -1 
  sign2 = order[order.index(pair[0])+1].even? ? 1 : -1 #(idxs[0][0] < idxs[1][0]) ? 1 : -1
#  p [rng.to_s, sign0, sign1, sign2] 
  sign = sign0*sign2
  sign *= -1 if (succs[1][1]==0)
  
  fac1 = box.map{|key, arr| [key, arr.select{|x| x <= pair[0] or pair[1] < x}]}.to_h
  fac2 = box.map{|key, arr| [key, arr.select{|x| pair[0] < x and x <= pair[1]}]}.to_h
  # contract fac1 & fac2
  #
  # shift numbers in fac1 & fac2
  factors = [fac1, fac2].map do |f|
    sorted = f.values.flatten.sort
    f.map{|key, arr| [key, arr.map{|v| sorted.index(v)}]}.to_h
  end
  # code --> word
  prewords = [fac1, fac2].map do |f|
    chars = (0..size/2).to_a.map{|i| f.map{|key, arr| key if arr.include?(2*i)}.join}
     Word.new(chars.join)
  end
  words = prewords.map do |ww|
    ww = ww.dup.contract 
    wcp = Group::Identity
    while (wcp.size < ww.size)
      ww.factors = wcp.factors unless wcp == Group::Identity
      wcp = ww.cyclic_permutation.contract
    end
    ww
  end
  {pair: pair, order: order, parity: parity, sign: sign, factors: factors, prewords: prewords, words: words}
end
printf("----------------------\n")

coprods.select{|h| h[:parity] == "odd"}.each do |h|
  strings = h[:factors].map{|f| f.values.map(&:to_s).join}

#  p "#{h[:pair]}"
  p "#{h[:order]}"
#  p "#{h[:parity]}"
#  p "#{h[:sign]}"
  p strings.join(" , ")
  p h[:prewords].join(" , ")
  p ((h[:sign]==1) ? "+" : "-") + h[:words].join("(x)")
  printf(".....\n")
end;nil

"abAB"
"[5, 0][3, 6][1, 4][7, 2][][][][][][][][]"
----------------------
"[3, 1, 4, 2]"
"[3, 0][4][1, 2][5][][][][][][][][] , [][1][][0][][][][][][][][]"
"aAB , b"
"+B(x)b"
.....
"[0, 1, 7, 2]"
"[0][][1][][][][][][][][][] , [3][1, 4][2][5, 0][][][][][][][][]"
"a , bAB"
"+a(x)A"
.....
"[5, 3, 6, 4]"
"[0][3, 4][1][5, 2][][][][][][][][] , [1][][0][][][][][][][][][]"
"abB , A"
"+a(x)A"
.....
"[5, 0, 6, 7]"
"[5, 0][3][1, 4][2][][][][][][][][] , [][0][][1][][][][][][][][]"
"abA , B"
"-b(x)B"
.....


### TODO
$[a_{1},b_{1}]$ の処理において、
* [ ] [5, 0, 6, 7] の sign が正しくないようだ。
* [ ] a(x)A の集計もおかしい。

In [ ]:
'''
printf("----------------------\n")
cprds = coprods.delete_if{|prd| prd[:parity] == "even"}

i = 0
while cprds.size > 0
  collection = [cprds.pop]
  collection += cprds.select{|other| other[:words] == collection[0][:words]}
#  p collection.map{|h| ((h[:sign]==1) ? "+" : "-") + "#{h[:words].map(&:show).join("(x)")}" }.join(" ")
  coeff = collection.map{|h| h[:sign]}.sum
  term = collection[0][:words].map(&:show).join("(x)")
  p case coeff 
    when 0 then "0"
    when 1 then term
    else coeff.to_s + "*" + term
  end

  cprds = cprds.delete_if{|h| h[:words] == collection[0][:words]}
end 
'''